In [1]:
import pandas as pd, numpy as np, re, os,datetime,calendar
    
################################备注########################################
#df_cleaned 时间列 = ['datetime','date','year','month','month_date','weekday','hour']
#df_simulated_hourly 时间列 = ['datetime','date','year','month','month_date','weekday','hour'] unit = ['Mwh']
#df_simulated_daily 时间列 = ['date','year','month','month_date','weekday'] unit = ['Gwh']
#df_simulated_monthly 时间列 = ['year','month'] unit = ['Gwh']
############################function#########################################
def create_folder(file_path,Type): #建立需要的文件夹
    out_path = file_path+Type+'\\'
    if not os.path.exists(out_path): #如果有了文件夹的话就直接pass掉
        os.mkdir(out_path)
    return out_path

def check_col(df,Type): #检查数据的现存列名是否一致 如果不一致就删掉多余的
    df_col = df.columns.tolist()
    if Type == 'hourly': #如果是simulated_hourly
        r_col = ['unit','datetime', 'date','year','month','month_date','weekday','hour','coal','oil','gas','nuclear','hydro','wind','solar','other','fossil','low.carbon','total.prod','coal.perc','oil.perc','gas.perc','nuclear.perc','hydro.perc','wind.perc','solar.perc','other.perc','fossil.perc','low.carbon.perc']
        df['unit'] = 'Mwh'
    if Type == 'daily':
        r_col = ['unit','date','year','month','month_date','weekday','coal','oil','gas','nuclear','hydro','wind','solar','other','fossil','low.carbon','total.prod','coal.perc','oil.perc','gas.perc','nuclear.perc','hydro.perc','wind.perc','solar.perc','other.perc','fossil.perc','low.carbon.perc']
        df['unit'] = 'Gwh'
    if Type == 'monthly':
        r_col = ['unit','year','month','coal','oil','gas','nuclear','hydro','wind','solar','other','fossil','low.carbon','total.prod','coal.perc','oil.perc','gas.perc','nuclear.perc','hydro.perc','wind.perc','solar.perc','other.perc','fossil.perc','low.carbon.perc']
        df['unit'] = 'Gwh'
    df = df[r_col]
    return df

def time_b_a(x,which): #根据which 选择得到所选日期的前which天或者后which天
    myday = datetime.datetime.strptime(x,'%Y-%m-%d')
    delta = datetime.timedelta(days=which)
    my_yestoday = myday + delta
    my_yes_time = my_yestoday.strftime('%Y-%m-%d')
    return my_yes_time
        
def time_info(df,date_name): #添加各种时间列
    df[date_name] = pd.to_datetime(df[date_name])
    df['year'] = df[date_name].dt.year
    df['month'] = df[date_name].dt.month
    df['month_date']  = df[date_name].dt.strftime('%m-%d')
    df['weekday'] = df[date_name].dt.day_name()
    if date_name != 'date':
        df['date'] = df[date_name].dt.strftime('%Y-%m-%d')
    
def check_date(df,date_name,f): #检查现存时间缺失值并填充
    df[date_name] = pd.to_datetime(df[date_name])
    real_date = pd.date_range(start=min(df[date_name]),end = max(df[date_name]),freq = f).tolist()
    df_date = df[date_name].drop_duplicates().tolist()
    missing_date = list(set(real_date)-set(df_date))
    for z in missing_date:
        df = df.append([{date_name:z}], ignore_index=True)
        df = df.sort_values(by = date_name).reset_index(drop = True)
    time_info(df,date_name)
    return df

#填充缺失值
def fill_null(df,date_name,Type):
    filling_result = []
    df_null = df[df.isna().any(axis=1)].reset_index(drop = True) #所有包含缺失值的行
    df_not_null = df[~df.isna().any(axis=1)].reset_index(drop = True) #所有不包含缺失值的行
    for x in df_null['year'].drop_duplicates().tolist(): #按年份循环
        df_temp = df_null[df_null['year'] == x].reset_index(drop = True) #按年份赋值新的df
        for y in df_temp['month'].drop_duplicates().tolist(): #新df中按月循环
            df_temp_month = df_temp[df_temp['month'] == y].reset_index(drop = True) #按月份赋值新的df
            month_date = calendar.monthrange(x,y)[1] #计算当月天数
            for z in df_mon.columns.tolist(): 
                if df_mon[z].dtype == float and z != 'total.gen': #如果这一列类型是float并且不为total时
                    df_temp_month[z] = df_mon[(df_mon['year'] == x) & (df_mon['month'] == y)][z].tolist()[0]/month_date
            filling_result.append(df_temp_month)
    df_missing = pd.DataFrame(np.concatenate(filling_result), columns = df_temp_month.columns)
    df = pd.concat([df_missing,df_not_null])
    df = df.sort_values(by = date_name).reset_index(drop = True)
    total_proc(df,unit = True)
    df = check_col(df,Type)
    return df

def total_proc(df,unit): #处理数据
    unit = unit
    fossil_list = ['coal','gas','oil']
    carbon_list = ['nuclear','hydro','wind','solar','other']
    perc_list = ['fossil','low.carbon']
    df[fossil_list] = df[fossil_list].astype(float)
    df[carbon_list] = df[carbon_list].astype(float)
    df['fossil'] = df[fossil_list].astype(float).sum(axis = 1)
    df['low.carbon'] = df[carbon_list].astype(float).sum(axis = 1)
    df['total.prod'] = df[perc_list].astype(float).sum(axis = 1)
    if unit == True:
        if 'hour' not in df.columns:#如果没有hour这一列 那就代表是daily或者monthly数据
            for z in df.columns.tolist():
                if df[z].dtype == float:
                    df[z] = df[z]/1000
    for y in df.columns.tolist():
        if df[y].dtype == float:
            if y != 'total.prod' and 'perc' not in y:
                df[y+'.perc'] = df[y]/df['total.prod']          
    df['unit'] = 'Gwh' #补充单位
    
def agg(df,date_name,path,Type,name,folder): #输出
    time_info(df,date_name)
    total_proc(df, unit = True)
    df = check_col(df,Type)
    if folder == True:
        out_path = create_folder(path,Type)
        out_file = out_path+name
    else:
        out_file = path+name
    df.to_csv(out_file, index = False, encoding = 'utf_8_sig')

In [ ]:
file_path = 'D:\\Python\\Work\\朱碧青\\Code_Store\\数据库\\data\\asia\\china\\'
in_path = file_path+'raw\\'
out_path_cleaned  = create_folder(file_path,'cleaned')
out_path_simulated = create_folder(file_path,'simulated')



In [9]:
test = pd.read_csv(r'D:\Python\Work\朱碧青\Data_Store\2022\02-24\occto.csv')
test_1 = pd.read_excel(r'D:\Python\Work\朱碧青\Data_Store\2022\02-24\utf-8-China_Power_calculation.xlsx')

In [10]:
test

,月日,時刻,需要,原子力,地熱,水力,火力,バイオマス,風力実績,風力抑制量,太陽光実績,太陽光抑制量,揚水
0,2019/1/1,0時,95523.2,9144,230,3791.0,78752.9,1266.9,1384.4,0.0,0.0,0.0,962
1,2019/1/1,1時,93129.1,9146,231,3881.0,76501.1,1265.9,1354.1,0.0,0.0,0.0,744
2,2019/1/1,2時,92090.7,9146,230,4109.0,75287.6,1262.9,1353.2,0.0,0.0,0.0,700
3,2019/1/1,3時,90745.0,9146,232,4421.0,73622.0,1263.9,1332.1,0.0,0.0,0.0,727
4,2019/1/1,4時,88693.4,9148,232,4281.0,72225.8,1262.9,1404.7,0.0,0.0,0.0,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25555,2021/11/30,19時,112255.0,7336,296,7762.9,90753.3,1618.8,2261.0,0.0,0.0,0.0,2057
25556,2021/11/30,20時,107730.8,7338,296,6714.9,87927.3,1618.8,2418.8,0.0,0.0,0.0,1265
25557,2021/11/30,21時,102241.6,7338,297,5758.9,83745.3,1621.8,2576.6,0.0,0.0,0.0,743
25558,2021/11/30,22時,96828.9,7339,295,5450.9,79650.1,1612.8,2636.1,0.0,0.0,0.0,-298
